In [67]:
import numpy as np
import cv2
from sklearn.neural_network import MLPRegressor
from matplotlib import pyplot as plt
import pandas as pd

### Read input data (depth camera)

In [ ]:
# read video frame
def dep_cam(dep_v_name):
    dep_v = cv2.VideoCapture(dep_v_name)
    ret, frame = dep_v.read()
    counter=0

    frame_count = int(dep_v.get(cv2.CAP_PROP_FRAME_COUNT))

    # frame_height = int(frame.shape[0]/5)
    # frame_width = int(frame.shape[1]/5)
    frame_height = frame.shape[0]
    frame_width = frame.shape[1]
    depth_frames = np.empty((frame_count, frame_height, frame_width))

    while(dep_v.isOpened()):
        ret, frame = dep_v.read()
        if ret == False:
            break
        if ret == True:
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            gray_frame = cv2.resize(gray_frame, (frame_width, frame_height), interpolation = cv2.INTER_AREA)
            depth_frames[counter] = gray_frame
            counter+=1

    dep_v.release()
    cv2.destroyAllWindows()
    return depth_frames

### Read input depth data (test folder 13, 14, 15, 16, 24 from Preliminary result v4)

In [ ]:
# read output angle data
# ignore
foldername = r'Test_Subject_Leo_Preliminary_v4/'
test_folder = ['test13', 'test14', 'test15', 'test16', 'test24' ]
test_num = ['13', '14', '15', '16', '24']
sub_name = foldername + test_folder[4]
dep_name = sub_name + r'/depth_processed_leo_test' + test_num[4] + '.avi';
depth =  dep_cam(dep_name)
for i in range(4):
    subfolder_name = foldername + test_folder[i]
    dep_video_name = subfolder_name + r'/depth_processed_leo_test' + test_num[i] + '.avi'
    dep_v_temp =  dep_cam(dep_video_name)
    depth = np.concatenate((dep_v_temp, depth))

### Read input depth data (test folder 24, 30, 31, 32, 33, 35 from Preliminary result v5)

In [ ]:
# read output angle data
foldername = r'Test_Subject_Leo/'
test_folder = ['test24', 'test30', 'test31', 'test32','test33', 'test35' ]
test_num = ['24', '30', '31', '32', '33', '35']
sub_name = foldername + test_folder[0]
dep_name = sub_name + r'/depth_processed_leo_test' + test_num[0] + '.avi';
depth =  dep_cam(dep_name)
for i in range(1, 6):
    subfolder_name = foldername + test_folder[i]
    dep_video_name = subfolder_name + r'/depth_processed_leo_test' + test_num[i] + '.avi'
    #print(dep_video_name)
    dep_v_temp =  dep_cam(dep_video_name)
    depth = np.concatenate((depth, dep_v_temp))

In [ ]:
print(depth.shape)

In [ ]:
# # plot one single frame
# plt.imshow(depth[5000], interpolation='nearest')
# plt.show()

### Read output data (angle) in test_folder 13, 14, 15, 16, 24 from Preliminary v4 

In [ ]:
subfolder_name = foldername + test_folder[4]
qtm_file = subfolder_name + r'/qtm_processed_leo_test' + test_num[4] + '_' + '11_15_2020' + '.txt'
theta_z = pd.read_csv(qtm_file)
theta_z = theta_z.iloc[:,0].values
for i in range(4):
    subfolder_name = foldername + test_folder[i]
#     if test_folder[i] == 'test24':
#         qtm_file = subfolder_name + r'/qtm_processed_leo_test' + test_num[i] + '_' + '11_15_2020' + '.txt'
#     else: 
    qtm_file = subfolder_name + r'/qtm_processed_leo_test' + test_num[i] + '_' + '11_11_2020' + '.txt'
    dataset_y =  pd.read_csv(qtm_file)
    theta_z_temp = dataset_y.iloc[:,0].values
    theta_z = np.concatenate((theta_z, theta_z_temp))

### Read output angle data (test folder 24, 30, 31, 32, 33, 35 from Preliminary result v5)

In [68]:
foldername = r'Test_Subject_Leo/'
test_folder = ['test24', 'test30', 'test31', 'test32','test33', 'test35' ]
test_num = ['24', '30', '31', '32', '33', '35']
subfolder_name = foldername + test_folder[0]
qtm_file = subfolder_name + r'/qtm_processed_leo_test' + test_num[0] + '_' + '11_15_2020' + '.txt'
theta_z = pd.read_csv(qtm_file)
theta_z = theta_z.iloc[:,0].values
for i in range(1,6):
    subfolder_name = foldername + test_folder[i]
    if test_folder[i] == 'test30' or test_folder[i] == 'test31':
        qtm_file = subfolder_name + r'/qtm_processed_leo_test' + test_num[i] + '_' + '11_24_2020' + '.txt'
    else: 
        qtm_file = subfolder_name + r'/qtm_processed_leo_test' + test_num[i] + '_' + '11_25_2020' + '.txt'
    dataset_y =  pd.read_csv(qtm_file)
    theta_z_temp = dataset_y.iloc[:,0].values
    theta_z = np.concatenate((theta_z, theta_z_temp))

In [ ]:
print(theta_z.shape)

### Split training and testing data

In [ ]:
depth_flatten = depth.reshape(19200, 82381).T

from sklearn.model_selection import train_test_split
X_train_sk, X_test_sk, y_train_sk, y_test_sk = train_test_split(depth_flatten, theta_z, test_size = 0.2, random_state = 0)

### Use scikit learn builtin neural network

In [ ]:
# use sklearn MLPRegressor
model_sk = MLPRegressor(hidden_layer_sizes = (200, ), activation = 'relu', learning_rate = 'constant',
                        random_state = 1, max_iter = 500, early_stopping = True, verbose = True).fit(X_train_sk, y_train_sk)


In [ ]:
from sklearn.metrics import mean_absolute_error
y_prediction_sk = model_sk.predict(X_test_sk)
mean_absolute_error(y_test_sk, y_prediction_sk)

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (20,5)
plt.plot(y_test)
plt.plot(y_prediction_sk,'--')

In [ ]:
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# sc.fit(X_train)
# Xtrain = sc.transform(X_train)
# Xtest = sc.transform(X_test)

In [ ]:
# use sklearn MLPRegressor
model_sk = MLPRegressor(hidden_layer_sizes = (100, ), activation = 'relu', learning_rate = 'constant',
                        random_state = 1, max_iter = 500, early_stopping = True, verbose = True).fit(X_train_sk, y_train_sk)

In [ ]:
from sklearn.metrics import mean_absolute_error
y_prediction_sk = model_sk.predict(X_test_sk)
mean_absolute_error(y_test_sk, y_prediction_sk)

### Using Tensorflow NN

In [ ]:
X_train_tf, X_test_tf, y_train_tf, y_test_tf = train_test_split(depth, theta_z, test_size = 0.2, random_state = 0)

In [ ]:
%load_ext tensorboard
import datetime, os

In [ ]:
import tensorflow as tf
#callback = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 10, restore_best_weights = True)

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(120, 160)),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dense(units=192, activation='relu'),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])


model.compile(optimizer='adam', 
              loss= 'mae',
              metrics=['mae'])
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
model.fit(X_train_tf, y_train_tf, epochs=3,callbacks = [tensorboard_callback], 
                    batch_size = 128, validation_data = (X_test_tf, y_test_tf))

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
# Get training and test loss histories
training_loss = history.history['loss']
test_loss = history.history['val_loss']

# Create count of the number of epochs
epoch_count = range(len(training_loss))

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
y_prediction_tf = model.predict(X_test_tf)
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (20,5)
plt.plot(y_test)
plt.plot(y_prediction_tf,'--')

In [ ]:
y_prediction_tf = model.predict(X_test_tf)
mean_absolute_error(y_test_tf, y_prediction_tf)

### Reducing the dimension of images

In [ ]:
# read video frame
def dep_cam_reduced(dep_v_name):
    dep_v = cv2.VideoCapture(dep_v_name)
    ret, frame = dep_v.read()
    counter=0

    frame_count = int(dep_v.get(cv2.CAP_PROP_FRAME_COUNT))

#     frame_height = int(frame.shape[0]/5)
#     frame_width = int(frame.shape[1]/5)
#     frame_height = frame.shape[0]
#     frame_width = frame.shape[1]
    frame_height = 30
    frame_width = 40
    depth_frames = np.empty((frame_count, frame_height, frame_width))

    while(dep_v.isOpened()):
        ret, frame = dep_v.read()
        if ret == False:
            break
        if ret == True:
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            gray_frame = cv2.resize(gray_frame, (frame_width, frame_height), interpolation = cv2.INTER_AREA)
            depth_frames[counter] = gray_frame
            counter+=1

    dep_v.release()
    cv2.destroyAllWindows()
    return depth_frames

In [ ]:
# read output depth images data
foldername = r'Test_Subject_Leo/'
test_folder = ['test24', 'test30', 'test31', 'test32','test33', 'test35' ]
test_num = ['24', '30', '31', '32', '33', '35']
sub_name = foldername + test_folder[0]
dep_name = sub_name + r'/depth_processed_leo_test' + test_num[0] + '.avi';
depth =  dep_cam_reduced(dep_name)
for i in range(1, 6):
    subfolder_name = foldername + test_folder[i]
    dep_video_name = subfolder_name + r'/depth_processed_leo_test' + test_num[i] + '.avi'
    #print(dep_video_name)
    dep_v_temp =  dep_cam_reduced(dep_video_name)/255.0
    depth = np.concatenate((depth, dep_v_temp))

In [ ]:
print(depth.shape)
print(theta_z.shape)

In [ ]:
X_train_tf, X_test_tf, y_train_tf, y_test_tf = train_test_split(depth, theta_z, test_size = 0.2, random_state = 0)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 3, restore_best_weights = True)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(30, 40)),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dense(units=192, activation='relu'),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

model.compile(optimizer='adam', 
              loss= 'mae',
              metrics=['mae'])

history = model.fit(X_train_tf, y_train_tf, epochs=250, callbacks = [callback], batch_size = 128, 
                    verbose = 1, validation_data = (X_test_tf, y_test_tf))

In [ ]:
# Get training and test loss histories
training_loss = history.history['loss']
test_loss = history.history['val_loss']

# Create count of the number of epochs
epoch_count = range(len(training_loss))

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
y_prediction = model.predict(X_test_tf)

import matplotlib as mpl
%matplotlib tk
mpl.rcParams['figure.figsize'] = (20,5)
plt.plot(y_test_tf)
plt.plot(y_prediction,'--')

In [ ]:
mean_absolute_error(y_test_tf, y_prediction)

In [ ]:
# try using tanh as activation function
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(120, 160)),
    tf.keras.layers.Dense(units=256, activation='tanh'),
    tf.keras.layers.Dense(units=192, activation='tanh'),
    tf.keras.layers.Dense(units=128, activation='tanh'),
    tf.keras.layers.Dense(1, activation = 'linear')
])

model.compile(optimizer='adam', 
              loss= 'mae',
              metrics=['mae'])

model.fit(X_train, y_train,epochs=50)

In [ ]:
y_prediction = model.predict(X_test)
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (20,5)
plt.plot(y_test)
plt.plot(y_prediction,'--')

In [ ]:
# try use sigmoid as activation function
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(120, 160)),
    tf.keras.layers.Dense(units=256, activation='sigmoid'),
    tf.keras.layers.Dense(units=192, activation='sigmoid'),
    tf.keras.layers.Dense(units=128, activation='sigmoid'),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', 
              loss= 'mae',
              metrics=['mae'])

model.fit(X_train, y_train,epochs=50)

### Comments:

###### Observation: Comparing to the NN that uses tanh and sigmoid activation function, the rectified linear unit fucntion seemed to have better performance (lower mae) when the layers number are the same. Also, the choice of metrics of model evaluation seemed to be important (mse is quite large compared to mae). But I wonder what happened to my plots ;(     

###### Also, right now I am using a dimension of 120 * 160, performing a dimensionality reduction could be helpful for extracting necessary features. A longer epoch time may also improve performance. Tried sklearn neural network MLPRegressor but does not work

### Add dropout to the NN

In [ ]:
X_train_tf, X_test_tf, y_train_tf, y_test_tf = train_test_split(depth, theta_z, test_size = 0.2, random_state = 0)

In [ ]:
import tensorflow as tf
callback = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 10, restore_best_weights = True)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(120, 160)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dense(units=192, activation='relu'),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])


model.compile(optimizer='adam', 
              loss= 'mae',
              metrics=['mae'])

history = model.fit(X_train_tf, y_train_tf, epochs=1000,callbacks = [callback], 
                    batch_size = 128, validation_data = (X_test_tf, y_test_tf))

In [ ]:
y_prediction = model.predict(X_test_tf)

import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (20,5)
plt.plot(y_test)
plt.plot(y_prediction,'--')

In [ ]:
# Get training and test loss histories
training_loss = history.history['loss']
test_loss = history.history['val_loss']

# Create count of the number of epochs
epoch_count = range(len(training_loss))

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

### NN with reduced dimensionality and dropout

In [ ]:
# dropout at visible layer
X_train_tf, X_test_tf, y_train_tf, y_test_tf = train_test_split(depth, theta_z, test_size = 0.2, random_state = 0)
import tensorflow as tf
callback = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 10, restore_best_weights = True)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(30, 40)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dense(units=192, activation='relu'),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])


model.compile(optimizer='adam', 
              loss= 'mae',
              metrics=['mae'])

history = model.fit(X_train_tf, y_train_tf, epochs=300,callbacks = [callback], 
                    batch_size = 128, validation_data = (X_test_tf, y_test_tf))

In [ ]:
# Get training and test loss histories
training_loss = history.history['loss']
test_loss = history.history['val_loss']

# Create count of the number of epochs
epoch_count = range(len(training_loss))

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
# dropout at both visible layer and hidden layer
X_train_tf, X_test_tf, y_train_tf, y_test_tf = train_test_split(depth, theta_z, test_size = 0.2, random_state = 0)
import tensorflow as tf
callback = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 10, restore_best_weights = True)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(30, 40)),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])


model.compile(optimizer='adam', 
              loss= 'mae',
              metrics=['mae'])

history = model.fit(X_train_tf, y_train_tf, epochs=300,callbacks = [callback], 
                    batch_size = 128, validation_data = (X_test_tf, y_test_tf))

In [ ]:
# Get training and test loss histories
training_loss = history.history['loss']
test_loss = history.history['val_loss']

# Create count of the number of epochs
epoch_count = range(len(training_loss))

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
y_prediction = model.predict(X_test_tf)

import matplotlib as mpl
%matplotlib tk
mpl.rcParams['figure.figsize'] = (20,5)
plt.plot(y_test_tf)
plt.plot(y_prediction,'--')
plt.legend(['Ground Truth', 'Prediction'])
plt.xlabel('Sample')
plt.ylabel('Angle (degree)')


In [ ]:
from sklearn.metrics import mean_absolute_error
y_pred = model.predict(X_test_tf)
mean_absolute_error(y_test_tf, y_pred)

### Discretize input signal (depth images)

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer
from tensorflow.keras import layers
from keras.utils import plot_model


In [ ]:
theta_z_temp = theta_z.reshape(-1,1)
trans = KBinsDiscretizer(n_bins=40, encode='onehot-dense', strategy='quantile')
data = trans.fit_transform(theta_z_temp)

In [ ]:
plt.plot(theta_z, 'r--')
plt.plot(data,'b-')
plt.legend(['Continuous', 'Discrete'])
plt.xlabel('Time')
plt.ylabel('Angle')
plt.show()

### Mulitple inputs in neural network

#### Read foce data

In [ ]:
subfolder_name = foldername + test_folder[0]
force_file = subfolder_name + r'/fcss_processed_leo_test' + test_num[0] + '_' + '11_15_2020' + '.txt'
force = pd.read_csv(force_file)
force = force.iloc[:,:].values
for i in range(1,6):
    subfolder_name = foldername + test_folder[i]
    if test_folder[i] == 'test30' or test_folder[i] == 'test31':
        force_file = subfolder_name + r'/fcss_processed_leo_test' + test_num[i] + '_' + '11_24_2020' + '.txt'
    else: 
        force_file = subfolder_name + r'/fcss_processed_leo_test' + test_num[i] + '_' + '11_25_2020' + '.txt'
    dataset_y =  pd.read_csv(force_file)
    force_temp = dataset_y.iloc[:,:].values
    force = np.concatenate((force, force_temp))

In [ ]:
print(force.shape)

#### Split the data into training and testing set

In [80]:
from sklearn.model_selection import train_test_split
depth_train, depth_test, force_train, force_test, theta_z_train, theta_z_test = train_test_split(depth, force, theta_z, test_size = 0.2, random_state = 0)

In [81]:
from tensorflow.keras import layers
input_depth = layers.Input(shape = (30, 40))
input_force = layers.Input(shape = (6, ))

# first branch
x = layers.Flatten(input_shape=(30, 40))(input_depth)
x = layers.Dense(units=128, activation='relu')(x)
x = layers.Dense(units=64, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(32, activation='relu')(x)
x = tf.keras.Model(inputs= input_depth, outputs = x)

# second branch
y = layers.Dense(units=128, activation='relu')(input_force)
y = layers.Dense(units=64, activation='relu')(y)
y = layers.Dense(32, activation='relu')(y)
y = layers.Dropout(0.2)(y)
y = tf.keras.Model(inputs= input_force, outputs = y)

# combine two branches
combined = layers.concatenate([x.output, y.output])

# regression to one output
z = layers.Dense(units=32, activation='relu')(combined)
z = layers.Dropout(0.2)(z)
z = layers.Dense(units=16, activation='relu')(combined)
z = layers.Dense(1, activation='linear')(z)
model = tf.keras.Model(inputs = [x.input, y.input], outputs = z)

#tf.keras.utils.plot_model(model, to_file = 'model.png')

In [82]:
model.compile(optimizer='adam', loss= 'mae', metrics=['mae'])

history = model.fit([depth_train, force_train],theta_z_train, verbose = 1, epochs = 300, batch_size = 128,
                   validation_data = ([depth_test, force_test], theta_z_test))

Epoch 1/300
515/515 [==============================] - 1s 2ms/step - loss: 8.3145 - mae: 8.3145 - val_loss: 7.2741 - val_mae: 7.2741
Epoch 2/300
515/515 [==============================] - 1s 2ms/step - loss: 7.1364 - mae: 7.1364 - val_loss: 6.7326 - val_mae: 6.7326
Epoch 3/300
515/515 [==============================] - 1s 2ms/step - loss: 6.5907 - mae: 6.5907 - val_loss: 6.2742 - val_mae: 6.2742
Epoch 4/300
515/515 [==============================] - 1s 2ms/step - loss: 6.1415 - mae: 6.1415 - val_loss: 5.9257 - val_mae: 5.9257
Epoch 5/300
515/515 [==============================] - 1s 2ms/step - loss: 5.8203 - mae: 5.8203 - val_loss: 5.6613 - val_mae: 5.6613
Epoch 6/300
515/515 [==============================] - 1s 2ms/step - loss: 5.6211 - mae: 5.6211 - val_loss: 5.5527 - val_mae: 5.5527
Epoch 7/300
515/515 [==============================] - 1s 1ms/step - loss: 5.5462 - mae: 5.5462 - val_loss: 5.5380 - val_mae: 5.5380
Epoch 8/300
515/515 [==============================] - 1s 1ms/step - 

In [ ]:
# Get training and test loss histories
training_loss = history.history['loss']
test_loss = history.history['val_loss']

# Create count of the number of epochs
epoch_count = range(len(training_loss))

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
y_prediction = model.predict([depth_test, force_test])

import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (20,5)
plt.plot(theta_z_test)
plt.plot(y_prediction,'--')
plt.legend(['Ground Truth', 'Prediction'])
plt.xlabel('Sample')
plt.ylabel('Angle (degree)')
plt.show()

In [ ]:
plt.scatter(y_prediction, theta_z_test)
plt.xlabel('Prediction angle')
plt.ylabel('Ground angle')
plt.show()

### Feed only force data to the NN

In [ ]:
import tensorflow as tf
from keras.utils.vis_utils import plot_model
callback = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 100, restore_best_weights = True)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

# plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
model.compile(optimizer='adam', 
              loss= 'mae',
              metrics=['mae'])

history = model.fit(force_train, theta_z_train, epochs=300,callbacks = [callback], 
                    batch_size = 128, validation_data = (force_test, theta_z_test))

In [ ]:
# Get training and test loss histories
training_loss = history.history['loss']
test_loss = history.history['val_loss']

# Create count of the number of epochs
epoch_count = range(len(training_loss))

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
y_prediction = model.predict(force_test)

import matplotlib as mpl
%matplotlib tk
mpl.rcParams['figure.figsize'] = (20,5)
plt.plot(theta_z_test)
plt.plot(y_prediction,'--')

### Retry Random Forest Regression

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

In [ ]:
# feature scaling
# fc = StandardScaler()
# sc_y = StandardScaler()

# force_train = fc.fit_transform(force_train)
# force_test = fc.transform(force_test)

# theta_z_train = theta_z_train.reshape(-1,1)
# theta_z_train = sc_y.fit_transform(theta_z_train)
# theta_z_train = theta_z_train.ravel()

regressor = RandomForestRegressor(n_estimators = 100, random_state = 2)
regressor.fit(force_train, theta_z_train)

#y_pred = sc_y.inverse_transform(regressor.predict(force_test))

In [ ]:
from sklearn.metrics import mean_absolute_error
y_pred = regressor.predict(force_test)
mean_absolute_error(theta_z_test, y_pred)

In [ ]:
regressor.score(force_test,theta_z_test)

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (15,10)
plt.plot(theta_z_test,'-')
plt.plot(y_pred,'--')

plt.title('Random Forest Regression')
plt.ylabel('Angle (deg)')
plt.xlabel('Time')
plt.show()

### Discretize output

In [69]:
from sklearn.preprocessing import KBinsDiscretizer
from tensorflow.keras import layers
from keras.utils import plot_model

In [70]:
theta_z = theta_z.reshape(-1, 1)
trans = KBinsDiscretizer(n_bins=200, encode='ordinal', strategy='uniform')
theta_z_disc = trans.fit_transform(theta_z)

In [74]:
import matplotlib as mpl
%matplotlib tk
mpl.rcParams['figure.figsize'] = (20,5)
plt.plot(theta_z, 'r--')
plt.plot(theta_z_disc,'b-')
plt.legend(['Continuous', 'Discrete'])
plt.xlabel('Time')
plt.ylabel('Angle')
plt.show()

In [72]:
error = theta_z_disc[0]-theta_z[0]

In [73]:
theta_z_disc = theta_z_disc - error

### Quantization for a single image

In [ ]:
# plot one single frame
cv2.imshow('image', depth[5000])

In [ ]:
from sklearn.cluster import MiniBatchKMeans
clt = MiniBatchKMeans(n_clusters = 4)
labels = clt.fit_predict(depth[5000])
quant = clt.cluster_centers_[labels]
cv2.imshow('image',quant)

### Quantization for depth images

In [75]:
def quantize(image):
    clt = MiniBatchKMeans(n_clusters = 4)
    labels = clt.fit_predict(image)
    quant_image = clt.cluster_centers_[labels]
    return quant_image

In [76]:
from sklearn.cluster import MiniBatchKMeans
depth_quantized = np.empty((depth.shape[0],depth.shape[1],depth.shape[2]))
for i in range(depth.shape[0]):
    depth_quantized[i] = quantize(depth[i])

In [ ]:
cv2.imshow('Image',depth_quantized[200])
cv2.imshow('Image', depth[200])

In [77]:
from sklearn.model_selection import train_test_split
depth_train, depth_test, force_train, force_test, theta_z_train, theta_z_test = train_test_split(depth_quantized, force, theta_z_disc, test_size = 0.2, random_state = 0)

In [78]:
import tensorflow as tf
from tensorflow.keras import layers
input_depth = layers.Input(shape = (30, 40))
input_force = layers.Input(shape = (6, ))

# first branch
x = layers.Flatten(input_shape=(30, 40))(input_depth)
x = layers.Dense(units=128, activation='relu')(x)
x = layers.Dense(units=64, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(32, activation='relu')(x)
x = tf.keras.Model(inputs= input_depth, outputs = x)

# second branch
y = layers.Dense(units=128, activation='relu')(input_force)
y = layers.Dense(units=64, activation='relu')(y)
y = layers.Dense(32, activation='relu')(y)
y = layers.Dropout(0.2)(y)
y = tf.keras.Model(inputs= input_force, outputs = y)

# combine two branches
combined = layers.concatenate([x.output, y.output])

# regression to one output
z = layers.Dense(units=32, activation='relu')(combined)
z = layers.Dropout(0.2)(z)
z = layers.Dense(units=16, activation='relu')(combined)
z = layers.Dense(1, activation='linear')(z)
model = tf.keras.Model(inputs = [x.input, y.input], outputs = z)

#tf.keras.utils.plot_model(model, to_file = 'model.png')

In [79]:
model.compile(optimizer='adam', loss= 'mae', metrics=['mae'])

history = model.fit([depth_train, force_train],theta_z_train, verbose = 1, epochs = 300, batch_size = 128,
                   validation_data = ([depth_test, force_test], theta_z_test))

Epoch 1/300
515/515 [==============================] - 1s 2ms/step - loss: 6.7041 - mae: 6.7041 - val_loss: 5.9654 - val_mae: 5.9654
Epoch 2/300
515/515 [==============================] - 1s 2ms/step - loss: 5.8554 - mae: 5.8554 - val_loss: 5.5684 - val_mae: 5.5684
Epoch 3/300
515/515 [==============================] - 1s 2ms/step - loss: 5.5466 - mae: 5.5466 - val_loss: 5.2943 - val_mae: 5.2943
Epoch 4/300
515/515 [==============================] - 1s 2ms/step - loss: 5.2659 - mae: 5.2659 - val_loss: 4.9893 - val_mae: 4.9893
Epoch 5/300
515/515 [==============================] - 1s 2ms/step - loss: 4.9619 - mae: 4.9619 - val_loss: 4.8505 - val_mae: 4.8505
Epoch 6/300
515/515 [==============================] - 1s 2ms/step - loss: 4.8140 - mae: 4.8140 - val_loss: 4.7246 - val_mae: 4.7246
Epoch 7/300
515/515 [==============================] - 1s 2ms/step - loss: 4.7189 - mae: 4.7189 - val_loss: 4.6052 - val_mae: 4.6052
Epoch 8/300
515/515 [==============================] - 1s 1ms/step - 

In [83]:
model.summary()

Model: "functional_23"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 30, 40)]     0                                            
__________________________________________________________________________________________________
flatten_4 (Flatten)             (None, 1200)         0           input_9[0][0]                    
__________________________________________________________________________________________________
input_10 (InputLayer)           [(None, 6)]          0                                            
__________________________________________________________________________________________________
dense_30 (Dense)                (None, 128)          153728      flatten_4[0][0]                  
______________________________________________________________________________________

In [85]:
depth_train, depth_test, force_train, force_test, theta_z_train, theta_z_test = train_test_split(depth, force, theta_z_disc, test_size = 0.2, random_state = 0)

In [86]:
import tensorflow as tf
from tensorflow.keras import layers
input_depth = layers.Input(shape = (30, 40))
input_force = layers.Input(shape = (6, ))

# first branch
x = layers.Flatten(input_shape=(30, 40))(input_depth)
x = layers.Dense(units=128, activation='relu')(x)
x = layers.Dense(units=64, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(32, activation='relu')(x)
x = tf.keras.Model(inputs= input_depth, outputs = x)

# second branch
y = layers.Dense(units=128, activation='relu')(input_force)
y = layers.Dense(units=64, activation='relu')(y)
y = layers.Dense(32, activation='relu')(y)
y = layers.Dropout(0.2)(y)
y = tf.keras.Model(inputs= input_force, outputs = y)

# combine two branches
combined = layers.concatenate([x.output, y.output])

# regression to one output
z = layers.Dense(units=32, activation='relu')(combined)
z = layers.Dropout(0.2)(z)
z = layers.Dense(units=16, activation='relu')(combined)
z = layers.Dense(1, activation='linear')(z)
model = tf.keras.Model(inputs = [x.input, y.input], outputs = z)

#tf.keras.utils.plot_model(model, to_file = 'model.png')
model.compile(optimizer='adam', loss= 'mae', metrics=['mae'])

history = model.fit([depth_train, force_train],theta_z_train, verbose = 1, epochs = 300, batch_size = 128,
                   validation_data = ([depth_test, force_test], theta_z_test))

Epoch 1/300
515/515 [==============================] - 1s 2ms/step - loss: 6.4728 - mae: 6.4728 - val_loss: 5.7229 - val_mae: 5.7229
Epoch 2/300
515/515 [==============================] - 1s 2ms/step - loss: 5.6974 - mae: 5.6974 - val_loss: 5.3898 - val_mae: 5.3898
Epoch 3/300
515/515 [==============================] - 1s 2ms/step - loss: 5.2905 - mae: 5.2905 - val_loss: 4.9091 - val_mae: 4.9091
Epoch 4/300
515/515 [==============================] - 1s 2ms/step - loss: 4.8886 - mae: 4.8886 - val_loss: 4.6913 - val_mae: 4.6913
Epoch 5/300
515/515 [==============================] - 1s 2ms/step - loss: 4.6739 - mae: 4.6739 - val_loss: 4.4580 - val_mae: 4.4580
Epoch 6/300
515/515 [==============================] - 1s 1ms/step - loss: 4.5751 - mae: 4.5751 - val_loss: 4.3827 - val_mae: 4.3827
Epoch 7/300
515/515 [==============================] - 1s 2ms/step - loss: 4.4253 - mae: 4.4253 - val_loss: 4.3367 - val_mae: 4.3367
Epoch 8/300
515/515 [==============================] - 1s 2ms/step - 

In [84]:
X_train_tf, X_test_tf, y_train_tf, y_test_tf = train_test_split(depth_quantized, theta_z_disc, test_size = 0.2, random_state = 0)
import tensorflow as tf
callback = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 10, restore_best_weights = True)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(30, 40)),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])


model.compile(optimizer='adam', 
              loss= 'mae',
              metrics=['mae'])

history = model.fit(X_train_tf, y_train_tf, epochs=300,callbacks = [callback], 
                    batch_size = 128, validation_data = (X_test_tf, y_test_tf))

Epoch 1/300
515/515 [==============================] - 1s 2ms/step - loss: 7.8862 - mae: 7.8862 - val_loss: 7.5961 - val_mae: 7.5961
Epoch 2/300
515/515 [==============================] - 1s 1ms/step - loss: 7.5531 - mae: 7.5531 - val_loss: 7.5408 - val_mae: 7.5408
Epoch 3/300
515/515 [==============================] - 1s 1ms/step - loss: 7.4656 - mae: 7.4656 - val_loss: 7.3679 - val_mae: 7.3679
Epoch 4/300
515/515 [==============================] - 1s 2ms/step - loss: 7.3643 - mae: 7.3643 - val_loss: 7.2519 - val_mae: 7.2519
Epoch 5/300
515/515 [==============================] - 1s 1ms/step - loss: 7.3073 - mae: 7.3073 - val_loss: 7.1640 - val_mae: 7.1640
Epoch 6/300
515/515 [==============================] - 1s 1ms/step - loss: 7.2836 - mae: 7.2836 - val_loss: 7.2576 - val_mae: 7.2576
Epoch 7/300
515/515 [==============================] - 1s 1ms/step - loss: 7.2511 - mae: 7.2511 - val_loss: 7.1030 - val_mae: 7.1030
Epoch 8/300
515/515 [==============================] - 1s 1ms/step - 